# LangChain サンプル 2: Retrieval

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### PDF ファイルをロードする DocumentLoader やベクターデータベースなどのインストール

In [3]:
!pip install pymupdf
!pip install spacy
!python3 -m spacy download ja_core_news_sm
!pip install chromadb==0.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 27.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 552.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 1.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 2.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 6.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 770.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 30.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 516.9 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### DocumentLoader と　Embed モデルによるベクターデータベース作成のサンプル

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader 
from langchain_aws import BedrockEmbeddings 
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.vectorstores import Chroma 

loader = PyMuPDFLoader("./AnyCompany.pdf") # PDF ファイルを読み込み
documents = loader.load()

text_splitter = SpacyTextSplitter(  # ドキュメント分割用に SpacyTextSplitterを初期化
    chunk_size=300, 
    pipeline="ja_core_news_sm"
)
splitted_documents = text_splitter.split_documents(documents)

embeddings = BedrockEmbeddings( # BedrockEmbeddings を初期化
    model_id = "amazon.titan-embed-text-v1" 
)

database = Chroma(  # Chromaを初期化
    persist_directory="./.data",  # データの保存先を指定
    embedding_function=embeddings  # Embed モデルを指定
)

database.add_documents(  # ドキュメントをデータベースに追加
    splitted_documents,  # 追加するドキュメント
)

print("データベースの作成が完了しました。") 

Created a chunk of size 382, which is longer than the specified 300


データベースの作成が完了しました。


### ベクターデータベースから類似性の高いドキュメントを取得するサンプル

In [ ]:
from langchain_aws import BedrockEmbeddings 
from langchain_community.vectorstores import Chroma 


embeddings = BedrockEmbeddings(
    model_id = "amazon.titan-embed-text-v1"  
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

documents = database.similarity_search("社員が結婚したときにの休暇は何日？") # データベースから類似度の高いドキュメントを取得
print(f"ドキュメントの数: {len(documents)}") # ドキュメントの数を表示

for document in documents:
    print(f"ドキュメントの内容: {document.page_content}") # ドキュメントの内容を表示

### ベクターデータベースから類似性の高いドキュメントを取得してモデルへ問い合わせを行うサンプル

In [ ]:
from langchain_aws import ChatBedrock                
from langchain_core.prompts.prompt import PromptTemplate  
from langchain_core.messages.human import HumanMessage 
from langchain_aws import BedrockEmbeddings  
from langchain_community.vectorstores import Chroma 


embeddings = BedrockEmbeddings(
    model_id = "amazon.titan-embed-text-v1"  # Bedrock では model_id にする
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

query = "社員が結婚したときにの休暇は何日？"

documents = database.similarity_search(query)

documents_string = "" # ドキュメントの内容を格納する変数を初期化

for document in documents:
    documents_string += f"""
---------------------------
{document.page_content}
""" # ドキュメントの内容を追加

prompt = PromptTemplate( # PromptTemplateを初期化
    template="""文章を元に質問に答えてください。 

文章: 
{document}

質問: {query}
""",
    input_variables=["document","query"] # 入力変数を指定
)

chat = ChatBedrock( #  ChatBedrockを初期化
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

result = chat.invoke([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])

print(result.content)


### ベクターデータベースから類似性の高いドキュメントを取得して　RetrievalQA を使用してモデルへ問い合わせを行うサンプル

In [ ]:
from langchain.chains import RetrievalQA  
from langchain_aws import ChatBedrock            
from langchain_aws import BedrockEmbeddings 
from langchain_community.vectorstores import Chroma 


chat = ChatBedrock(model_id = "meta.llama3-8b-instruct-v1:0")  

embeddings = BedrockEmbeddings(
     model_id = "amazon.titan-embed-text-v1"  
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

retriever = database.as_retriever() # データベースをRetrieverに変換

qa = RetrievalQA.from_llm(         #  RetrievalQAを初期化
    llm=chat,                      #  Chat modelsを指定
    retriever=retriever,           #  Retrieverを指定
    return_source_documents=True   # レスポンスにソースドキュメントを含めるかを指定
)


result = qa.invoke("社員が結婚したときにの休暇は何日？")

print(result["result"]) # レスポンスを表示

print(result["source_documents"]) # ソースドキュメントを表示